In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
torch.manual_seed(1337)

batch_size = 64
block_size = 256

lr = 3e-4

max_iters = 5000

eval_iters = 500
eval_interval = 200

num_embeddings = 384
num_heads = 6
num_layer = 6

dropout = 0.2

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
with open("../data/shakespeare.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [4]:
#Vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(vocab_size)
print(chars)


65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
str_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_str = {i:ch for i, ch in enumerate(chars)}
encoder = lambda s: [str_to_int[char] for char in s]
decoder = lambda l: "".join([int_to_str[i] for i in l])

In [6]:
#encode, save it as torch tensor
data = torch.tensor(encoder(text), dtype=torch.long)

In [7]:
split_percentage = int(0.9*len(data))

train_data = data[:split_percentage]
valid_data = data[split_percentage:]

In [8]:
def get_batch(split):
    data = train_data if split == "train" else valid_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    x,  y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [13]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(num_embeddings, head_size, bias=False)
        self.query = nn.Linear(num_embeddings, head_size, bias=False)
        self.value = nn.Linear(num_embeddings, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
    
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2, -1) * C ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.projection = nn.Linear(num_embeddings, num_embeddings)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = torch.cat([head(x) for head in self.heads], dim=-1)
        out = self.dropout(self.projection(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, num_embeddings):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(num_embeddings, num_embeddings*4),
            nn.ReLU(),
            nn.Linear(num_embeddings*4, num_embeddings),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, num_embeddings, num_heads):
        super().__init__()
        self.self_attention_heads = MultiHeadAttention(num_heads, num_embeddings // num_heads)
        self.feedforward = FeedForward(num_embeddings)
        self.layer_norm1 = nn.LayerNorm(num_embeddings)
        self.layer_norm2 = nn.LayerNorm(num_embeddings)
    
    def forward(self, x):
        x = x + self.self_attention_heads(self.layer_norm1(x))
        x = x + self.feedforward(self.layer_norm2(x))
        return x


In [14]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, num_embeddings)
        self.position_embedding_table = nn.Embedding(block_size, num_embeddings)
        # self.self_attention_heads = MultiHeadAttention(4, num_embeddings//4)
        # self.feedforward = FeedForward(num_embeddings)
        self.blocks = nn.Sequential(*[Block(num_embeddings, num_heads=num_heads) for _ in range(num_layer)])
        self.final_layer_norm = nn.LayerNorm(num_embeddings)
        self.lm_head = nn.Linear(num_embeddings, vocab_size)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape

        token_embedding = self.token_embedding_table(idx)
        position_embedding = self.position_embedding_table(torch.arange(T, device=device))

        x = token_embedding + position_embedding
        # x = self.self_attention_heads(x)
        # x = self.feedforward(x)
        x = self.blocks(x)
        x = self.final_layer_norm(x)

        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]

            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = BigramLanguageModel()
model = model.to(device)


In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

In [16]:
for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"Step {iter}: Train loss {losses["train"]:.4f}, Val loss {losses["val"]:.4f}")


    xb, yb = get_batch("train")

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

Step 0: Train loss 4.2990, Val loss 4.2986
Step 200: Train loss 2.4242, Val loss 2.4469
Step 400: Train loss 2.1804, Val loss 2.2251
Step 600: Train loss 1.9285, Val loss 2.0318
Step 800: Train loss 1.7393, Val loss 1.8905
Step 1000: Train loss 1.6219, Val loss 1.7961
Step 1200: Train loss 1.5375, Val loss 1.7208
Step 1400: Train loss 1.4738, Val loss 1.6668
Step 1600: Train loss 1.4247, Val loss 1.6291
Step 1800: Train loss 1.3849, Val loss 1.5976
Step 2000: Train loss 1.3515, Val loss 1.5672
Step 2200: Train loss 1.3260, Val loss 1.5608
Step 2400: Train loss 1.3024, Val loss 1.5409
Step 2600: Train loss 1.2748, Val loss 1.5229
Step 2800: Train loss 1.2585, Val loss 1.5164
Step 3000: Train loss 1.2337, Val loss 1.5055
Step 3200: Train loss 1.2169, Val loss 1.4944
Step 3400: Train loss 1.2042, Val loss 1.4926
Step 3600: Train loss 1.1831, Val loss 1.4795
Step 3800: Train loss 1.1714, Val loss 1.4825
Step 4000: Train loss 1.1555, Val loss 1.4829
Step 4200: Train loss 1.1409, Val loss 1.

In [19]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decoder(model.generate(context, 500)[0].tolist()))


They're but respect, or their prove the doction:
The other holy endure's down, double two firmless
Have both of him feetly try like shoul;
For he is tender unserved: no but what widows with we
Hath might accesstomass to join them; and not it,
If leaching Camillo her for my heart of follows,
imaginately makes the need at fly to my
Of fairly, and let hie jest ithell.
You have saiden, consed with viled feast, some conceived
that; no more on the Tybalne bed some conference;
for dies now, forwards hi


In [20]:
MODEL_PATH = "language_model.pt"
torch.save(model.state_dict(), MODEL_PATH)
print(f"Model saved to {MODEL_PATH}")


Model saved to language_model.pt
